In [1]:
import pandas as pd

df = pd.read_csv('F:\Programming Stuff\IR\ActionsBhai\Reviews_2.csv')
df.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,107834,B0002EXXD0,A2PL0US7RN67G,"Jennifer ""Jennifer""",0,0,4,1202688000,Love this stuff!,Great sugar replacement. Used this on my kids...
1,255154,B001EO6424,A2XR7BQ3BJ7MYB,Aggie,3,5,5,1253664000,Smooth and good.,"This is my tea of choice, and it has always be..."
2,33370,B007JFMH96,A2HIL1IWWD2WEA,Stephen Cabral,0,0,2,1345680000,Dissapointed,I thought this cookie was extremely dry. It ne...
3,108718,B000EXKS14,AKMEY1BSHSDG7,J. Arena,0,0,5,1209513600,Makes any meal special!,It's so very worth it to keep a stash of Lipto...
4,171273,7310172001,A1UVLNNXPKEO04,Mr Dune,0,0,5,1319587200,Crack For Dogs!,My dogs love this stuff! They will do anything...


In [2]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install spacy

Note: you may need to restart the kernel to use updated packages.


In [4]:
import spacy
import string
import nltk
from nltk.stem import PorterStemmer
import re
nlp = spacy.load("en_core_web_sm")
stop_words = spacy.lang.en.stop_words.STOP_WORDS
stemmer = PorterStemmer()

def cleaning(s):
    s = str(s)
    s = re.sub('\s\W',' ',s)
    s = re.sub('\W,\s',' ',s)
    s = re.sub("\d+", "", s)
    s = re.sub('\s+',' ',s)
    s = re.sub('[!@#$_]', '', s)
    s = s.replace("co","")
    s = s.replace("https","")
    s = s.replace("[\w*"," ")
    return s

def preprocess_text(text):
    text = str(text)
    text = cleaning(text)
    doc = nlp(text)
    tokens = [token.lemma_.lower() for token in doc if not token.is_punct]
    tokens = [token for token in tokens if token not in stop_words]
    tokens = [stemmer.stem(token) for token in tokens]
    preprocessed_text = ' '.join(tokens)

    return preprocessed_text

In [5]:
df.dropna()
df.columns

Index(['Id', 'ProductId', 'UserId', 'ProfileName', 'HelpfulnessNumerator',
       'HelpfulnessDenominator', 'Score', 'Time', 'Summary', 'Text'],
      dtype='object')

In [6]:
df = df.sample(frac=0.2, random_state=42)

In [7]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(df, test_size=0.25, random_state=42)

In [8]:
# a = df.shape[0]
# print(a)
# a = a//6

In [9]:
# reduced_df = df.sample(frac=0.0167, random_state=42)

In [10]:
# reduced_df.shape

In [11]:
# reduced_df.to_csv('Reviews_2.csv', index=False)

In [12]:
# reduced_df.dropna()
# reduced_df.shape

In [13]:
train_df['Summary_preProcessed'] = train_df['Summary'].apply(preprocess_text)
train_df['Text_preProcessed'] = train_df['Text'].apply(preprocess_text)
train_df['Training'] = train_df['Text_preProcessed']  + ' TL;DR ' + train_df['Summary_preProcessed']

In [14]:
test_df['Summary_preProcessed'] = test_df['Summary'].apply(preprocess_text)
test_df['Text_preProcessed'] = test_df['Text'].apply(preprocess_text)

In [15]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

In [16]:
train_data_list = train_df['Training'].tolist()

In [17]:
print(train_data_list)

['ginger kind thing eat regularli big dose like flavor sip drink mixtur think good rest year thing know troubl soon open packet power come think packet mean entir gallon littl cup.<br /><br />make drink simple- add packet content cup add hot water drink readi smell great wait tri good god flavor bold suggest buy drink love ginger mean crazi like taz mutter word probabl like stuff bare detect hint lemon flavor featur ginger sucker punch mouth relentlessli big ginger root blend tastes.<br /><br />even 5 hour tri tast have eat dinner lot water milk definit shot TL;DR want drink ginger root chanc', 'coffe good prefer grind bean good vanilla flavor fresh buy sale worth price favorit coffe buy sale TL;DR tasti good vanilla flavor', 'honestli < href="http://www.amazon.com gp product b001lggh54">the switch watermelon strawberri flavor</a > terrible.<br /><br />it 4 main ingredi < br />1 carbon water < br />2 juic concentr strawberri watermelon grape acerola appl < br />3 natur flavor < br />4 

In [18]:
# tokenizer.pad_token = tokenizer.eos_token

# input_encodings = tokenizer(texts, truncation=True, padding=True, max_length=512)
# label_encodings = tokenizer(summaries, truncation=True, padding=True, max_length=128)

In [19]:
# print(input_encodings.keys())

In [20]:
# import torch
# from sklearn.model_selection import train_test_split

# input_ids = input_encodings['input_ids']
# attention_mask = input_encodings['attention_mask']
# labels = label_encodings['input_ids']

# train_input_ids, test_input_ids, train_attention_mask, test_attention_mask, train_labels, test_labels = train_test_split(
#     input_ids, attention_mask, labels, test_size=0.25, random_state=42
# )

In [21]:
from torch.utils.data import Dataset, DataLoader
import torch

class GPT2Dataset(torch.utils.data.Dataset):
    def __init__(self, data, tokenizer, max_len):
        self.data = data
        self.tokenizer = tokenizer
        self.tokenizer.pad_token_id = self.tokenizer.eos_token_id
        # self.padder = tokenizer.eos_token
        self.max_len = max_len
        # self.input_ids = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, id):
        data = self.data[id]

        tokenized_data = self.tokenizer.encode(data+self.tokenizer.eos_token)

        if len(tokenized_data)<self.max_len:
            tokenized_data+= [self.tokenizer.pad_token_id]*(self.max_len-len(tokenized_data))

        if len(tokenized_data)>self.max_len:
            tokenized_data = tokenized_data[:self.max_len]

        labels = tokenized_data[1:]+[self.tokenizer.pad_token_id]
        # self.input_ids = tokenized_data

        return {
            'input_ids': torch.tensor(tokenized_data),
            'labels': torch.tensor(labels)
        }

In [22]:
train_dataset = GPT2Dataset(train_data_list, tokenizer, max_len= 265)
# test_dataset = GPT2Dataset(test_input_ids, test_attention_mask, test_labels)

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
# test_loader = DataLoader(test_dataset, batch_size=4, shuffle=True)

In [24]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [27]:
import torch
from torch.optim import Adam

optimizer = Adam(model.parameters(), lr=1e-6)

model.train()
for epoch in range(3):
    total_loss = 0
    for batch in train_loader:
        optimizer.zero_grad()

        input_ids = batch['input_ids'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids, labels=labels)
        loss = outputs.loss
        total_loss+= loss
        loss.backward()
        optimizer.step()
        
    print(f"Epoch {epoch}, Loss: {total_loss/len(train_loader)}")


Epoch 0, Loss: 1.9571560621261597
Epoch 1, Loss: 1.8124713897705078
Epoch 2, Loss: 1.750462532043457


In [ ]:
# from transformers import AdamW

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model.to(device)

# optimizer = AdamW(model.parameters(), lr=0.01)

# for epoch in range(3):
#     model.train()
#     total_loss = 0
#     for batch in train_loader:
#         inputs, labels = batch['input_ids'].to(device), batch['input_ids'].to(device)
#         outputs = model(inputs, labels=labels)
#         loss = outputs.loss
#         total_loss += loss.item()

#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()

#     print(f"Epoch {epoch+1}, Loss: {total_loss/len(train_loader)}")


In [28]:
# Save the model and tokenizer
model.save_pretrained("./saved_model")
tokenizer.save_pretrained("./saved_model")

('./saved_model\\tokenizer_config.json',
 './saved_model\\special_tokens_map.json',
 './saved_model\\vocab.json',
 './saved_model\\merges.txt',
 './saved_model\\added_tokens.json')

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

model = GPT2LMHeadModel.from_pretrained("./saved_model")
tokenizer = GPT2Tokenizer.from_pretrained("./saved_model")

In [ ]:
import locale
print(locale.getpreferredencoding())

In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [41]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch


model.eval()

def generate_summary(text):
    input_ids = tokenizer.encode(str(text)+" TL;DR", return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = model.generate(input_ids, max_length=len(input_ids[0])+200,num_beams=10,     # Set the number of beams for beam search
                length_penalty=3.0,  # Set the length penalty for beam search
                repetition_penalty=3.0,  # Set the repetition penalty for beam search
                early_stopping=False)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

test_data = input()
generated_summaries = [generate_summary(test_data)]

print(generated_summaries[0])

print("dummy")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Great sugar replacement.  Used this on my kids cereal and they didn't even know the difference! TL;DR I love it!!!

Reviewed By Date Rating Strength Flavoring Taste Room Note Emeritus Account (28392) 2009-12-01 Medium None Detected Medium Pleasant to Tolerable This is one of those blends that has a lot going for it. It's not as sweet as some other tobaccos out there, but if you're looking for something with a bit more sweetness then this will be right up your alley. Nobody has rated this review yet.
dummy


In [ ]:
# import numpy as np
# from rouge_score import rouge_scorer

# result = input()
# actual_summaries = [str(result)]
# scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
# rouge_results = [scorer.score(target, pred) for target, pred in zip(actual_summaries, generated_summaries)]

# def average_rouge_scores(rouge_results):
#     rouge1_f1 = np.mean([score['rouge1'].fmeasure for score in rouge_results])
#     rouge2_f1 = np.mean([score['rouge2'].fmeasure for score in rouge_results])
#     rougeL_f1 = np.mean([score['rougeL'].fmeasure for score in rouge_results])
#     return {'rouge1_f1': rouge1_f1, 'rouge2_f1': rouge2_f1, 'rougeL_f1': rougeL_f1}

# average_scores = average_rouge_scores(rouge_results)
# print("Average ROUGE Scores:", average_scores)


In [42]:
import numpy as np
from rouge_score import rouge_scorer

result = input()
actual_summaries = [str(result)]

scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
rouge_results = [scorer.score(target, pred) for target, pred in zip(actual_summaries, generated_summaries)]

def average_rouge_scores(rouge_results):
    avg_scores = {}

    for rouge_type in ['rouge1', 'rouge2', 'rougeL']:
        avg_precision = np.mean([score[rouge_type].precision for score in rouge_results])
        avg_recall = np.mean([score[rouge_type].recall for score in rouge_results])
        avg_fmeasure = np.mean([score[rouge_type].fmeasure for score in rouge_results])

        avg_scores[f'{rouge_type}_precision'] = avg_precision
        avg_scores[f'{rouge_type}_recall'] = avg_recall
        avg_scores[f'{rouge_type}_fmeasure'] = avg_fmeasure
    return avg_scores

average_scores = average_rouge_scores(rouge_results)

print("Average ROUGE Scores:")
for score_type, value in average_scores.items():
    print(f"{score_type}: {value:.4f}")

Average ROUGE Scores:
rouge1_precision: 0.0213
rouge1_recall: 0.6667
rouge1_fmeasure: 0.0412
rouge2_precision: 0.0000
rouge2_recall: 0.0000
rouge2_fmeasure: 0.0000
rougeL_precision: 0.0213
rougeL_recall: 0.6667
rougeL_fmeasure: 0.0412
